Utilisation nltk pour nettoyage de données. Le but de ce traitement est de corriger les fautes possibles de décodation.

In [ ]:
STOPWORDS = stopwords.words('french')
from nltk.stem import *
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french", ignore_stopwords = False)
def clearing_word(word):
    word = re.sub('\x8e', 'é', word)
    word = re.sub('\x88', 'à', word)
    word = re.sub('\x9d', 'ù', word)
    word = re.sub('\x8f', 'è', word)
    word = re.sub('\x9e', 'û', word)
    word = re.sub('\x90', 'ê', word)
    word = re.sub('\x99', 'ô', word)
    word = re.sub('\x94', 'î', word)
   # word = re.sub('\x8f', 'è', word)
    word = re.sub('\x8d', 'ç', word)
    word = re.sub('õ', '', word)
    word = re.sub('Ê', '', word)
    word = re.sub('[?,.,!, \,,  %]', '', word)
    if word == 'û' or word == 'v' or word == 'é':
        word = ''
    if word =="2017êles":
        word = "2017"
    if "ênox" in word:
        word ="nox"
    return word


def clear_line(pharse):
    """
    Arg: string
    Return: string
    """
    dict_manuel = dict_manuelle()
    pharse_ =[]
    for words in pharse.strip().strip('?').strip('.').strip('!').lower().split(' '):
        for word in words.split('-'):
            for word_ in word.split('\''):
                word_ = clearing_word(word_)
                word_ = stemmer.stem(word_)
        #if word_ not in stop_word and len(word_) >1:       
        pharse_.append(word_)  
        for i in range(len(pharse_)):
            for line in dict_manuel:
                if pharse_[i] in line:
                    pharse_[i] = line[0]
    return ' '.join(pharse_)

def stemmer_line(line):
    return ' '.join([stemmer.stem(word) for word in line.split()])

def prepareData(PAIRS):
    pairs_trains, pairs_tests = [], []
   # PAIRS.extend(pairs1)
    index_train = random.sample(range(len(PAIRS)), int(len(PAIRS)*0.80))
    for i in range(len(PAIRS)):
        if i in index_train:
            pairs_trains.append(PAIRS[i])
        else:
            pairs_tests.append(PAIRS[i])
    print("Read %s sentence pairs of training set" % len(pairs_trains))
    print("Read %s sentence pairs of test set" % len(pairs_tests))
    return pairs_trains, pairs_tests

def get_all_convos(stopwords, file):
    
    convos = []
    questions_to_print = []
    liste_file = [file]
    list_of_word = []
    for file in liste_file:
        with open(file) as f:
            i=0
            for line in f:
                if i%2==0:
                    question = str(line)
                    if '++++' in question:
                         question = question[8:]
                    question_ = question.strip().strip('!').strip('?').strip('.')
                    questions_to_print.append(question_)
                    question = clear_line(question)
                else:
                    answer = str(line)
                    if '++++' in answer:
                        answer = answer[8:].strip()
                    convos.append([question, answer])
                i+=1
        f.close()
        good_index = []
        convos_ = []
        for i in range(len(convos)):
            if convos[i][0] not in convos_:
                convos_.append(convos[i][0])
                good_index.append(i)
            
    
    return [convos[i] for i in good_index], [questions_to_print[i] for i in good_index]
